In [4]:
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm



tqdm.pandas()

In [5]:
from googleapiclient.discovery import build
from bs4 import BeautifulSoup
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

api_key = 'YOUR_API_KEY'

from googleapiclient.discovery import build
from google.oauth2.credentials import Credentials

# Authenticate with your Google Cloud Platform project
#creds = Credentials.from_authorized_user_file('your_credentials.json')
youtube = build('youtube', 'v3',  developerKey=api_key)

# Specify the playlist ID
playlist_id = 'PL4sOqnpg9DQd501qYIrKyKLQMdbKlmXU5'


/usr/local/python/3.12.1/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [21]:

# Request playlist items
request = youtube.playlistItems().list(
    part='snippet,contentDetails',
    playlistId=playlist_id,
    maxResults=50
)
response = request.execute()

# Create a list to store video information
video_data = []

for item in tqdm(response['items']):
    video_id = item['contentDetails']['videoId']
    title = item['snippet']['title']
    thumbnail_url = item['snippet']['thumbnails']['default']['url']
    published_at = item['snippet']['publishedAt']

    channel_id = item['snippet']['videoOwnerChannelId']
    channel_title = item['snippet']['videoOwnerChannelTitle']

    # Request video statistics
    request = youtube.videos().list(
        part='statistics',
        id=video_id
    )
    response = request.execute()

    # Extract statistics
    video_stats = response['items'][0]['statistics']
    view_count = video_stats['viewCount']
    like_count = video_stats['likeCount']
    try:
        dislike_count = video_stats['dislikeCount']
    except KeyError:
        dislike_count = 0  # Or you can set a default value like 0

    video_data.append([video_id, title,  thumbnail_url, published_at, channel_id, channel_title, view_count, like_count, dislike_count])

# Create a DataFrame from the list
df = pd.DataFrame(video_data, columns=['Video ID', 'Title', 'Thumbnail URL', 'Published At', 'Channel ID', 'Channel Title', 'View Count', 'Like Count', 'Dislike Count'])

100%|██████████| 24/24 [00:02<00:00,  8.88it/s]


In [22]:
df

,Video ID,Title,Thumbnail URL,Published At,Channel ID,Channel Title,View Count,Like Count,Dislike Count
0,SRXH9AbT280,The Emptiness Machine (Official Music Video) -...,https://i.ytimg.com/vi/SRXH9AbT280/default.jpg,2024-11-21T00:29:22Z,UCZU9T1ceaOgwfLRq7OKFU4Q,Linkin Park,65871614,1746087,0
1,kivUsDGWojU,Two Faced (Official Music Video) - Linkin Park,https://i.ytimg.com/vi/kivUsDGWojU/default.jpg,2024-11-21T00:29:34Z,UCZU9T1ceaOgwfLRq7OKFU4Q,Linkin Park,11394160,606893,0
2,5FrhtahQiRc,Heavy Is The Crown ft. Linkin Park (Official M...,https://i.ytimg.com/vi/5FrhtahQiRc/default.jpg,2024-11-21T00:29:41Z,UC2t5bjwHdUX4vM2g8TRDq5g,League of Legends,63687895,1124422,0
3,HhM2OCLfxgo,LINKIN PARK LIVE IN SÃO PAULO BRAZIL 11/15/24 ...,https://i.ytimg.com/vi/HhM2OCLfxgo/default.jpg,2024-11-21T00:29:49Z,UCoxv8AchXrFUAG18Z41kTjw,Alexsander Alves,52630,2783,0
4,fSHoePrnmMw,Over Each Other (Official Music Video) - Linki...,https://i.ytimg.com/vi/fSHoePrnmMw/default.jpg,2024-11-21T00:29:58Z,UCZU9T1ceaOgwfLRq7OKFU4Q,Linkin Park,11365409,450287,0
5,EjDM7eGMAlA,Linkin Park - From Zero - Live - Allianz Parqu...,https://i.ytimg.com/vi/EjDM7eGMAlA/default.jpg,2024-11-21T00:30:04Z,UCn2dVkuGINywLqzV1k0MViw,WG.88,54176,2981,0
6,7i4s-NXfkPk,Numb (Live) - Linkin Park,https://i.ytimg.com/vi/7i4s-NXfkPk/default.jpg,2024-11-21T00:30:23Z,UCZU9T1ceaOgwfLRq7OKFU4Q,Linkin Park,8072828,236301,0
7,_f9b0NB5o4E,Cut The Bridge (Official Audio Visualizer) - L...,https://i.ytimg.com/vi/_f9b0NB5o4E/default.jpg,2024-11-21T00:30:51Z,UCZU9T1ceaOgwfLRq7OKFU4Q,Linkin Park,1756248,69964,0
8,g4HzHarOX60,"LPTV FROM ZERO: Making of ""Two Faced"" Music Vi...",https://i.ytimg.com/vi/g4HzHarOX60/default.jpg,2024-11-21T00:31:00Z,UCZU9T1ceaOgwfLRq7OKFU4Q,Linkin Park,207365,17723,0
9,Ip0jJACsE_g,Good Things Go (Official Audio Visualizer) - L...,https://i.ytimg.com/vi/Ip0jJACsE_g/default.jpg,2024-11-21T00:31:05Z,UCZU9T1ceaOgwfLRq7OKFU4Q,Linkin Park,1421598,63749,0


In [23]:
df.to_csv('linkin_park_youtube.csv', sep=',')

In [24]:
# Function to clean HTML with additional validation
def clean_html(comment):
    # Check if the comment is non-empty and likely HTML
    if isinstance(comment, str) and "<" in comment and ">" in comment:
        soup = BeautifulSoup(comment, 'html.parser')
        return soup.get_text()
    # If it's not HTML, return the comment as-is
    return comment

In [25]:
# Request playlist items
request = youtube.playlistItems().list(
    part='snippet,contentDetails',
    playlistId=playlist_id,
    maxResults=50
)
response = request.execute()

# Create a list to store video information and comments
video_comments_data = []

for item in  tqdm(response['items']):
    video_id = item['contentDetails']['videoId']

    # Request video comments
    next_page_token = None
    while True:
        request = youtube.commentThreads().list(
            part='snippet',
            videoId=video_id,
            maxResults=100,
            pageToken=next_page_token
        )
        response = request.execute()

        for comment in response['items']:
            comment_text = clean_html(comment['snippet']['topLevelComment']['snippet']['textDisplay'])
            commenter_name = comment['snippet']['topLevelComment']['snippet']['authorDisplayName']
            comment_published_at = comment['snippet']['topLevelComment']['snippet']['publishedAt']

            video_comments_data.append([video_id, comment_text, commenter_name, comment_published_at])

        next_page_token = response.get('nextPageToken')

        if not next_page_token:
            break

# Create a DataFrame from the list of video comments
df_comments = pd.DataFrame(video_comments_data, columns=['Video ID', 'Comment', 'Commenter', 'Comment Datetime'])

print(df_comments)


100%|██████████| 24/24 [09:51<00:00, 24.63s/it]

           Video ID                                            Comment  \
0       SRXH9AbT280                   Suddently my mind goes into 2000   
1       SRXH9AbT280  Linkin Park lifer here. Huge fan since the fir...   
2       SRXH9AbT280  No, she doesn't sound like Chester.\r But she ...   
3       SRXH9AbT280  Man scientology can pay for a lot can&#39;t they?   
4       SRXH9AbT280  I don&#39;t connect with the melodies and line...   
...             ...                                                ...   
279961  wTG5aRpYIoA  Linkin Park are getting better and better ever...   
279962  wTG5aRpYIoA  Linkin Park are getting better and better ever...   
279963  wTG5aRpYIoA  Fess up, how many times have you watched this ...   
279964  wTG5aRpYIoA                            Nice one. Thanks mate🤘🏼   
279965  wTG5aRpYIoA  When I pretend everything is what I want it to...   

                              Commenter      Comment Datetime  
0                       @achmadamri4967  2024-1

In [26]:
df_comments.head(20)

,Video ID,Comment,Commenter,Comment Datetime
0,SRXH9AbT280,Suddently my mind goes into 2000,@achmadamri4967,2024-11-21T04:33:46Z
1,SRXH9AbT280,Linkin Park lifer here. Huge fan since the fir...,@michaelgilroy5077,2024-11-21T04:13:52Z
2,SRXH9AbT280,"No, she doesn't sound like Chester.\r But she ...",@fernandodelgadomelesio7250,2024-11-21T04:01:17Z
3,SRXH9AbT280,Man scientology can pay for a lot can&#39;t they?,@bloodedge555,2024-11-21T04:00:01Z
4,SRXH9AbT280,I don&#39;t connect with the melodies and line...,@nicolasdejesusmoretagonzal5203,2024-11-21T03:56:18Z
5,SRXH9AbT280,"If I have to say the best song of this year, i...",@marsagadri,2024-11-21T03:49:24Z
6,SRXH9AbT280,This message is a warning to all Quintessons. ...,@redhood982,2024-11-21T03:28:22Z
7,SRXH9AbT280,God bless chester bennington,@Starboi_gassin,2024-11-21T03:20:33Z
8,SRXH9AbT280,I’m so glad I pre-ordered this album.,@claireconover,2024-11-21T02:43:50Z
9,SRXH9AbT280,Pathetic to be honest. Why not just start a ne...,@DmofukinLee,2024-11-21T02:35:19Z


In [27]:
df_comments.tail(50)

,Video ID,Comment,Commenter,Comment Datetime
279916,wTG5aRpYIoA,The London fans nailed Crawling! Awesome job. ...,@greghalter934,2024-10-05T03:04:16Z
279917,wTG5aRpYIoA,15% concentrated pineapple smells,@MagicalSkyWizard,2024-10-05T00:40:00Z
279918,wTG5aRpYIoA,Some girl in the crowd going beast mode during...,@MagicalSkyWizard,2024-10-05T00:15:26Z
279919,wTG5aRpYIoA,😍,@LomiroLopublico,2024-10-04T03:34:30Z
279920,wTG5aRpYIoA,1:29:05 das antigas Friendly Fire é a que mais...,@gillprock,2024-10-04T00:04:37Z
279921,wTG5aRpYIoA,Emili jest niesamowita ❤️❤️❤️🤘,@przemysawzurek6418,2024-10-03T19:11:48Z
279922,wTG5aRpYIoA,Waiting for the end. What a brilliant job they...,@paulinnanjing,2024-10-03T16:55:49Z
279923,wTG5aRpYIoA,"Add a show to australia already, ill sell my f...",@elvislannon,2024-10-03T13:05:51Z
279924,wTG5aRpYIoA,bad was the band has lost a lot in live shows ...,@ukyoOkashi,2024-10-02T18:59:37Z
279925,wTG5aRpYIoA,cheers to the gir giving her all during the em...,@xavivila9839,2024-10-02T17:23:48Z


In [36]:
df_comments.shape

(279966, 5)

In [ ]:
# Load the sentiment analysis pipeline using a pre-trained RoBERTa model
model = "cardiffnlp/twitter-roberta-base-sentiment"

sentiment_pipeline = pipeline(
    "sentiment-analysis", 
    model= model, 
    tokenizer= model,
    truncation=True  # Ensure inputs are truncated to the model's maximum length
)

from tqdm import tqdm
tqdm.pandas()  # Initialize tqdm for Pandas

# Function to analyze sentiment and map it to human-readable labels
def analyze_and_map_sentiment(comment):
    try:
        # Pass the comment to the pipeline with truncation
        result = sentiment_pipeline(comment[:512])[0]  # Limit to 512 characters
        label = result['label']
        
        # Map the label to human-readable sentiment
        if label == 'LABEL_2':  # RoBERTa's labels: 0 = negative, 1 = neutral, 2 = positive
            return 'positive'
        elif label == 'LABEL_1':
            return 'neutral'
        elif label == 'LABEL_0':
            return 'negative'
        return 'unknown'
    except Exception as e:
        print(f"Error processing comment: {comment}\nError: {e}")
        return None

# Apply the function with a progress bar
df_comments['Sentiment'] = df_comments['Comment'].progress_apply(analyze_and_map_sentiment)


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Error processing comment: Бляяяяя аж поделиться впечатлениями нескем - всё моё окружение слушает какое то гавно😢. Даже те кто говорят что слушают рок вообще любой и те пиздят и от силы за всю дорогу слушали только те легендарные песни которые слышали все. Но я ахренеть как рад и ахренеть как впечатлен как тогда, в 2004 году, когда зашел к девке в квартиру с которой тащился как маньяк😂. Тогда предвкушая свой третий или 2 секс в жизни я подошел к муз центру и схватил первый попавшийся диск и воткнул в машину, и заиграла эта жесть на всю жизнь изменивший мои муз вкусы❤. Немедленно вернувшись к диску я на всю жизнь запомнил эту обложку. И вот 20 лет уже если у меня в руках пульт от чего то способное воспроизвести музыку сразуууууу же METEOOOOORAAAAA🤘🤘🤘🤘🤘🤘🤘
Error: The expanded size of the tensor (549) must match the existing size (514) at non-singleton dimension 1.  Target sizes: [1, 549].  Tensor sizes: [1, 514]
Error processing comment: Так радостно слышать любимую группу вновь. Да, Честе

In [1]:
df_comments.to_csv('linkin_park_youtube_comments.csv', sep=',')

NameError: name 'df_comments' is not defined